### Depending on how you set up your env you might need to run the following pip

In [1]:
#!pip install dash-bootstrap-components
#!pip install jupyter_dash

### dependencies

In [17]:
#import dependencies
import plotly.express as px
import pandas as pd
import numpy as np
import json
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_table import DataTable

#removes the deprecated warning
import warnings
warnings.filterwarnings('ignore')

### json

In [18]:
#load geojson file
with open('canada_provinces.geojson') as f:
    provinces = json.load(f)

### df for choropleth map (plot 1)

In [19]:
#load raw csv data
cov_df = pd.read_csv("covid19-download.csv")

#we only need the provincial data
cov_df = cov_df [ cov_df ['prname'] != 'Canada' ]
cov_df = cov_df [ cov_df ['prname'] != 'Repatriated travellers' ]

#relevant variables are number of cases (numtotal)
#number of new cases (numtoday), number of deaths (numdeaths), number of recovery (numrecover)
col_of_interest = ['prname', 'date', 'numtotal', 'numtoday', 'numdeaths', 'numrecover']

#select relevant columns via col_of_interest
cov_df = cov_df[[c for c in cov_df.columns if c in col_of_interest]]

#convert date from object to datetime
cov_df['date'] =  pd.to_datetime(cov_df['date'])

#Since the data from Covid-19 cases stopped being updated for some provinces in the last month, use the
#dates between January 1, 2020, and February 28, 2022, to create the dashboard.
start_day = '01/01/2020'
end_day = '28/02/2022'
start_day = pd.to_datetime(start_day, infer_datetime_format=True)
end_day = pd.to_datetime(end_day, infer_datetime_format=True)
cov_df = cov_df[cov_df['date'].between(start_day, end_day)]

#rename columns
cov_df = cov_df.rename(columns={"prname": "Province",
                                "numdeaths": "Total Deaths",
                                "numtotal": "Total Cases",
                                "numrecover": "Total Recovery",
                                "numtoday": "New Cases",
                                "date": "Date"})

#dataframe for choropleth map
cov_df_eom = cov_df[pd.to_datetime(cov_df["Date"]).dt.is_month_end]
cov_df_eom['Date'] = cov_df_eom['Date'].dt.strftime('%Y-%m-%d')

### df for lineplot (plot 4)

In [20]:
#load raw csv data
vax_df = pd.read_csv("vaccination-coverage-map.csv")

#only keep columns of interest
col_of_int = ['week_end', 'prename', 'numtotal_fully']
vax_df = vax_df[[c for c in vax_df.columns if c in col_of_int]]

#remove canada
vax_df = vax_df [ vax_df ['prename'] != 'Canada' ]

#convert date from object to datetime
vax_df['week_end'] =  pd.to_datetime(vax_df['week_end'])

#Since the data from Covid-19 cases stopped being updated for some provinces in the last month, use the
#dates between January 1, 2020, and February 28, 2022, to create the dashboard.
start_day = '01/01/2020'
end_day = '28/02/2022'
start_day = pd.to_datetime(start_day, infer_datetime_format=True)
end_day = pd.to_datetime(end_day, infer_datetime_format=True)
vax_df = vax_df[vax_df['week_end'].between(start_day, end_day)]

#convert datetime to string to create keyed_df
vax_df['week_end'] = vax_df['week_end'].dt.strftime('%Y-%m-%d')

#create keyed vax_df
vax_df["key"] = vax_df["week_end"] + vax_df["prename"]
vax_df_keyed = vax_df

#convert datetime to string to create keyed_df
cov_df['Date'] = cov_df['Date'].dt.strftime('%Y-%m-%d')

#concat and create dateprovince key
cov_df["key"] = cov_df["Date"] + cov_df["Province"]
keyed_cov_df = cov_df

merged_df = keyed_cov_df.merge(vax_df_keyed, on='key', how="inner")

col_of_int = ['Province',
              'Date',
              'Total Deaths',
              'Total Cases',
              'Total Recovery',
              'New Cases',
              'numtotal_fully']

merged_df_filtered = merged_df[[c for c in merged_df.columns if c in col_of_int]]

### df for vert bar, horiz bar, and table (plot 2,3,5)

In [21]:
#load raw csv data
cov_df = pd.read_csv("covid19-download.csv")

#we only need the provincial data
cov_df = cov_df [ cov_df ['prname'] != 'Canada' ]
cov_df = cov_df [ cov_df ['prname'] != 'Repatriated travellers' ]

#relevant variables are number of cases (numtotal)
#number of new cases (numtoday), number of deaths (numdeaths), number of recovery (numrecover)
col_of_interest = ['prname', 'date', 'numtotal', 'numtoday', 'numdeaths', 'numrecover']

#select relevant columns via col_of_interest
cov_df = cov_df[[c for c in cov_df.columns if c in col_of_interest]]

#convert date from object to datetime
cov_df['date'] =  pd.to_datetime(cov_df['date'])

#Since the data from Covid-19 cases stopped being updated for some provinces in the last month, use the
#dates between January 1, 2020, and February 28, 2022, to create the dashboard.
start_day = '01/01/2020'
end_day = '28/02/2022'
start_day = pd.to_datetime(start_day, infer_datetime_format=True)
end_day = pd.to_datetime(end_day, infer_datetime_format=True)
cov_df = cov_df[cov_df['date'].between(start_day, end_day)]

#rename columns
cov_df = cov_df.rename(columns={"prname": "Province",
                                "numdeaths": "Total Deaths",
                                "numtotal": "Total Cases",
                                "numrecover": "Total Recovery",
                                "numtoday": "New Cases",
                                "date": "Date"})

### Dash Setup

In [22]:
#create an indicator array to go into the dropdown
pivot_df = cov_df.T.reset_index()
indicator_array = pivot_df['index'].unique()

indicator_array = np.delete(indicator_array, 0)
indicator_array = np.delete(indicator_array, 0)
indicator_array

array(['Total Deaths', 'Total Cases', 'Total Recovery', 'New Cases'],
      dtype=object)

In [24]:
app = JupyterDash(__name__, 
                external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    
    html.H1('DATA 622: Final COVID-19 Data Plotly Dash', style={'textAlign': 'center'}),
    
    html.Br(),
    
    dbc.Row([
        dbc.Col([dbc.Label("Select a COVID-19 Metric:", style={'textAlign': 'left', 'fontSize': '25px', 'fontWeight': 'bold'}),
                 dcc.Dropdown(id='id_indicator', value="Total Cases",
                              options=[{'label':ind, 'value':ind } for ind in indicator_array])], md=12, lg=12),]),
    
    dbc.Card(
        dbc.CardBody([
            
            dbc.Row([
                
                dbc.Col([dbc.Row([dcc.Graph("plot_two_vertbar")]),
                    dbc.Row([dcc.Graph("plot_three_horizbar")])], width=4),
                
                dbc.Col([
                    dcc.Graph("plot_one_choropleth")], width=4),
                
                dbc.Col([
                    dbc.Row([dcc.Graph("plot_four_line")]),
                    dbc.Row([html.Div(id='plot_five_table')])], width=4),
                    
            ], align='center')])), 
            ])

# 1) CHOROPLETH #######################################################################################################
@app.callback(Output(component_id="plot_one_choropleth", component_property="figure"),
              Input(component_id="id_indicator", component_property="value"))

def plot_choropleth(indicator):

    #To create a column that is the date converted to a string, put this line before the fig assignment line: #***
    cov_df_eom['Date'] = cov_df_eom['Date'].apply(lambda x: str(x))

    fig1 = px.choropleth(cov_df_eom,
                         geojson=provinces,
                         featureidkey='properties.name',
                         locations='Province',
                         color=indicator,
                         color_continuous_scale='Blues',
                         title=f'Geographic Dispersion of {indicator} at End of Month',
                         height=950,
                         scope='north america',
                         animation_frame='Date')

    fig1.layout.geo.showframe = False
    fig1.update_layout(coloraxis_colorbar_title_text = f'{indicator}')
    fig1.layout.geo.landcolor = 'white'
    fig1.update_geos(fitbounds="locations")

    return fig1

# 2) VERT BAR ###########################################################################################################
@app.callback(Output(component_id="plot_two_vertbar", component_property="figure"),
              Input(component_id="id_indicator", component_property="value"))

def plot_vertbar(indicator):

    fig2 = px.bar(cov_df,
                  x="Date",
                  y=indicator,
                  title=f'{indicator} vs. Time',
                  color='Province',
                  height=500,
                  color_discrete_sequence=px.colors.qualitative.G10)
    
    
    fig2.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

    return fig2


# 3) HORIZ BAR ##########################################################################################################
@app.callback(Output(component_id="plot_three_horizbar", component_property="figure"),
              Input(component_id="id_indicator", component_property="value"))

def plot_horizbar(indicator):

    cov_df_horizbar = cov_df[cov_df['Date'] == '2022-02-28'].sort_values(indicator, ascending=False)

    fig3 = px.bar(cov_df_horizbar,
                  x=indicator,
                  y='Province',
                  title=f'{indicator} by Province',
                  color='Province',
                  orientation='h',
                  height=500,
                  log_x=True,
                  color_discrete_sequence=['#4f8fc6']*len(cov_df_horizbar),
                  text_auto='.2s')
    
    fig3.update_layout(showlegend=False)
    
    return fig3

# 4) LINE PLOT ##########################################################################################################
@app.callback(Output(component_id="plot_four_line", component_property="figure"),
              Input(component_id="id_indicator", component_property="value"))

def plot_line(indicator):

    fig4 = px.line(merged_df_filtered,
                   x="numtotal_fully",
                   log_x=True,
                   log_y=True,
                   title=f'{indicator} vs. Number People of Fully Vaccinated',
                   y=indicator,
                   height=500,
                   color='Province',
                   labels={"numtotal_fully": "Number of People Fully Vaccinated"},
                   color_discrete_sequence=px.colors.qualitative.G10)
    
    return fig4

# 5) TABLE ##############################################################################################################
@app.callback(Output(component_id='plot_five_table',component_property='children'),
              Input(component_id='id_indicator', component_property='value'))

def plot_table(indicator):
    
    list_of_int = ['Province', 'Date']
    list_of_int.append(indicator)
    cov_df_filtered = cov_df[[c for c in cov_df.columns if c in list_of_int]]
    cov_df_filtered['Date'] = cov_df_filtered['Date'].dt.strftime('%d-%m-%Y')

    table5 = dbc.Col([DataTable(data=cov_df_filtered.to_dict('records'),
                               columns=[{'name': col, 'id': col} for col in cov_df_filtered.columns],
                               style_header={'whiteSpace': 'normal'}, #allow the header to overflow for long text
                               fixed_rows={'headers': True}, #When rolling, allows the headers to be in a fix position.
                               virtualization=True, #allow virtualization, app load a portion of the data (for long talbles)
                               style_as_list_view=True,
                               style_table={'height': '400px'})], lg =12)
    
    return table5

# Run app and display result inline in the notebook
app.run_server(mode='external', width='90%', debug=False)

Dash app running on http://127.0.0.1:8050/
